In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

11490434/11490434 [==============================] - 1s 0us/step


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
def Create_model(learning_rate=0.001,batch_size=32,optimizer="Adam"):
  model=Sequential([Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [16]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[reduce_lr])

Epoch 1/10
1875/1875 [==============================] - 64s 33ms/step - loss: 0.1287 - accuracy: 0.9612 - val_loss: 0.0510 - val_accuracy: 0.9843 - lr: 0.0010
Epoch 2/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0436 - accuracy: 0.9870 - val_loss: 0.0284 - val_accuracy: 0.9906 - lr: 0.0010
Epoch 3/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0297 - accuracy: 0.9906 - val_loss: 0.0375 - val_accuracy: 0.9880 - lr: 0.0010
Epoch 4/10
1875/1875 [==============================] - 58s 31ms/step - loss: 0.0212 - accuracy: 0.9931 - val_loss: 0.0385 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 5/10
1875/1875 [==============================] - 55s 30ms/step - loss: 0.0164 - accuracy: 0.9946 - val_loss: 0.0325 - val_accuracy: 0.9904 - lr: 0.0010
Epoch 6/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.0223 - val_accuracy: 0.9929 - lr: 2.0000e-04
Epoch 7/10
1875/1875 [====================

In [17]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f%%' % (accuracy * 100))

313/313 [==============================] - 3s 9ms/step - loss: 0.0274 - accuracy: 0.9934
Accuracy: 99.34%


In [ ]:
#Text Classification
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt

# Load IMDb dataset
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Preprocess the text data
max_len = 500
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)


In [19]:
def create_model(learning_rate=0.001, optimizer='adam'):
    model = Sequential([
        Embedding(num_words, 32),
        SimpleRNN(32),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [ ]:
# Hyperparameters to tune
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'optimizer': ['adam', 'sgd']
}

# Create KerasClassifier for GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=5, batch_size=32, verbose=0)

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)


In [ ]:
# Get best parameters
best_params = grid_result.best_params_
learning_rate = best_params['learning_rate']
optimizer = best_params['optimizer']

# Create and compile the model
model = create_model(learning_rate=learning_rate, optimizer=optimizer)

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test), callbacks=[reduce_lr])

# Evaluate the model on test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')
